In [ ]:
# Simulation mean
mean_mvgauss = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# HOI params
n_trials = 10000
n_roi = 12
n_times=700
cov=np.zeros((13,13))

# Factor loadings



#FIRST MODULE OF BRAIN REGIONS 0 oinfo

lambx = np.sqrt(0.99)
lamby = np.sqrt(0.7)
lambz = np.sqrt(0.3)
lambs = np.sqrt(0.5)


# Full factor matrix L
L = np.array([lambx, lamby, lambz])[np.newaxis]

#We fix theta_yz in such a way that O(R1,R2,R3)=0
theta_yz = -0.148 

# Noise covariances theta
theta = np.diagflat(1 - L**2)
theta += np.diagflat([0, theta_yz], 1) + np.diagflat([0, theta_yz], -1)

#The covariance matrix for the three brain regions
cov0 = L * L.T + theta

#SECOND MODULE OF BRAIN REGIONS oinfo>0

lambx = np.sqrt(0.99)
lamby = np.sqrt(0.7)
lambz = np.sqrt(0.3)
lambs = np.sqrt(0.5)


# Full factor matrix L
L = np.array([lambx, lamby, lambz])[np.newaxis]


#We fix theta_yz in such a way that O(R1,R2,R3)<0
theta_yz = 0.1

# Noise covariances theta
theta = np.diagflat(1 - L**2)
theta += np.diagflat([0, theta_yz], 1) + np.diagflat([0, theta_yz], -1)

#The covariance matrix for the three brain regions
cov_red= L * L.T + theta

#SECOND MODULE OF BRAIN REGIONS oinfo>0

lambx = np.sqrt(0.99)
lamby = np.sqrt(0.7)
lambz = np.sqrt(0.3)
lambs = np.sqrt(0.1)


# Full factor matrix L
L = np.array([lambx, lamby, lambz])[np.newaxis]

#We fix theta_yz in such a way that O(R1,R2,R3)>0
theta_yz = -0.32

# Noise covariances theta
theta = np.diagflat(1 - L**2)
theta += np.diagflat([0, theta_yz], 1) + np.diagflat([0, theta_yz], -1)

#The covariance matrix for the three brain regions
cov_syn= L * L.T + theta

cov[:3,:3]=cov_red
cov[3:6,3:6]=cov_syn
cov[6:9,6:9]=cov_red
cov[9:12,9:12]=cov_syn


#Imposing the relationships with the behaviour

L = np.array([lambx, lamby, lambz, 0.7])[np.newaxis]

theta = np.diagflat(1 - L**2) 

#For Red

theta_zs = 0.05

theta += np.diagflat([0, 0, theta_zs], 1) + np.diagflat([0, 0, theta_zs], -1) 

cov_ = L * L.T + theta

cov[:3,12]=cov_[3,:3]
cov[12,:3]=cov_[3,:3]

#For Syn

L = np.array([0.99, 0.7,0.3, 0.1])[np.newaxis]

theta_zs = -0.28
theta = np.diagflat(1 - L**2) 
theta += np.diagflat([0, 0, theta_zs], 1) + np.diagflat([0, 0, theta_zs], -1) 


cov_ = L * L.T + theta

cov[12,3:6]=cov_[3,:3]
cov[3:6,12]=cov_[3,:3]

cov[12,12]=1

plt.imshow(cov, aspect='auto')
plt.colorbar()
plt.xticks([])
plt.yticks([])
plt.savefig('cov_matrix_12roi_1beh.png', dpi=600)

print(np.linalg.det(cov))

x_brain = np.random.multivariate_normal(mean_mvgauss[:-1], cov[:-1,:-1], size=n_trials, check_valid='warn', tol=1e-8)

x=np.zeros((n_trials, n_roi+1, n_times))

for i in range(n_times):
    x[:,:,i] = np.random.multivariate_normal(mean_mvgauss, cov, size=n_trials, check_valid='warn', tol=1e-8)

beh = x[:,12,:]

#hanning_time = np.repeat(np.repeat(np.hanning(n_times)[np.newaxis,:], n_roi, axis=0)[np.newaxis,:,:], n_trials, axis=0)
#hanning_time_invers = 1-np.repeat(np.repeat((np.hanning(n_times))[np.newaxis,:], n_roi, axis=0)[np.newaxis, :, :],  n_trials, axis=0)

#simulated_data_brain = x_brain * hanning_time_invers + hanning_time * x[:,:12,:]
simulated_data_brain = x[:,:12,:]
simulated_data=x
#simulated_data=np.concatenate((simulated_data_brain, np.repeat(beh[:,np.newaxis, np.newaxis], n_times, axis=2)),axis=1)
